In [8]:
import numpy as np
import tensorflow as tf
from matplotlib import pylab as plt

In [4]:
data = np.load("burgers_data.npy")

In [5]:
data

array([[[ 1.00000000e+00,  1.10000000e+00,  1.20000000e+00, ...,
          2.80000000e+00,  2.90000000e+00,  3.00000000e+00],
        [ 9.00000000e-01,  9.90000000e-01,  1.08000000e+00, ...,
          2.52000000e+00,  2.61000000e+00,  2.70000000e+00],
        [ 8.18181818e-01,  9.00000000e-01,  9.81818182e-01, ...,
          2.29090909e+00,  2.37272727e+00,  2.45454545e+00],
        ...,
        [ 5.62500000e-01,  6.18750000e-01,  6.75000000e-01, ...,
          1.57500000e+00,  1.63125000e+00,  1.68750000e+00],
        [ 5.29411765e-01,  5.82352941e-01,  6.35294118e-01, ...,
          1.48235294e+00,  1.53529412e+00,  1.58823529e+00],
        [ 5.00000000e-01,  5.50000000e-01,  6.00000000e-01, ...,
          1.40000000e+00,  1.45000000e+00,  1.50000000e+00]],

       [[ 9.00000000e-01,  8.10000000e-01,  7.20000000e-01, ...,
         -7.20000000e-01, -8.10000000e-01, -9.00000000e-01],
        [ 1.00000000e+00,  9.00000000e-01,  8.00000000e-01, ...,
         -8.00000000e-01, -9.00000000e

In [7]:
Nsol,Nt,Nx = data.shape

In [10]:
data[5,0:2,:]

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]])

In [34]:
Nsamp = Nsol*(Nt-1)*(Nx-2)
u_in = np.empty((Nsamp,3),dtype=np.float32)
u_out = np.empty((Nsamp,1),dtype=np.float32)
idx = 0
for i in range(Nsol):
    for j in range(Nt-1):
        for k in range(1,Nx-1):
            u_in[idx,:] = data[i,j,k-1:k+2]
            u_out[idx] = data[i,j+1,k]
            idx+=1

In [35]:
u_out

array([[0.99      ],
       [1.08      ],
       [1.17      ],
       ...,
       [0.9416199 ],
       [0.97082037],
       [0.99160796]], dtype=float32)

In [36]:
u_in.shape

(1881, 3)

In [37]:
import torch

In [38]:
model = torch.nn.Sequential(
    torch.nn.Linear(3,20),
    torch.nn.ReLU(),
    torch.nn.Linear(20,20),
    torch.nn.ReLU(),
    torch.nn.Linear(20,10),
    torch.nn.ReLU(),
    torch.nn.Linear(10,1),
)
loss = torch.nn.MSELoss(reduction='sum')

In [40]:
to_u_in = torch.tensor(u_in)
to_u_out = torch.tensor(u_out)

In [41]:
loss(model(to_u_in),to_u_out)

tensor(2082.9565, grad_fn=<MseLossBackward>)

In [46]:
learning_rate = 1e-4
for t in range(1000):
    y_pred = model(to_u_in)
    lo = loss(y_pred, to_u_out)
    if t % 100 == 99:
        print(t, lo.item())
    model.zero_grad()
    lo.backward()
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

99 30.34462547302246
199 31.341533660888672
299 29.20819091796875
399 26.668506622314453
499 24.04515838623047
599 20.459543228149414
699 22.217260360717773
799 29.770471572875977
899 23.68661117553711
999 18.62187957763672
